# Preprocessing - Making the data ready for machine learning consumption

in here we will preprocess the data, we mainly do encoding on the data

# Importing data and library

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# for printing data more prettier
# https://docs.python.org/3/library/pprint.html
from pprint import pprint

In [3]:
df = pd.read_csv('data/data-ml-nonpreprocess.csv')
df.head()

,taskId,taskCreatedTime,taskStatus,cod.amount,UserVar.branch_dest,UserVar.receiver_city,UserVar.weight,UserVar.branch_origin,isCOD,timeDiffMinutes
0,4fe3b237c832ca4841a2,2022-11-01 13:17:26+00:00,done,685000.0,SRG,"BATANG ,KAB BATANG",13.000,CGK,1,29.066667
1,08a4da25256affae8446,2022-11-01 01:41:07+00:00,done,53500.0,MGL,"PURWODADI,PURWOREJO",1.300,CGK,1,232.683333
2,2ff0dc469826158b7684,2022-11-01 01:41:07+00:00,done,179500.0,MGL,"PURWODADI,PURWOREJO",3.000,CGK,1,300.833333
3,331c172c2b383f774328,2022-11-01 01:41:07+00:00,done,31815.0,MGL,"PURWODADI,PURWOREJO",0.625,CGK,1,577.200000
4,a9d53fa96c80baee8b23,2022-11-01 01:41:07+00:00,done,144562.0,MGL,"BAGELEN,PURWOREJO",3.000,CGK,1,130.700000


In [4]:
df.isna().sum()

taskId                     0
taskCreatedTime            0
taskStatus                 0
cod.amount                 0
UserVar.branch_dest        0
UserVar.receiver_city      0
UserVar.weight             0
UserVar.branch_origin      0
isCOD                      0
timeDiffMinutes          742
dtype: int64

# Encoding branch

we are going to do one hot encoding, but before that, as we know from the pandas profiling result, that branch origin dan branch destination has different amount of branch

In [5]:
print(df['UserVar.branch_origin'].unique().shape)
np.sort(df['UserVar.branch_origin'].unique())

(59,)


array(['AMI', 'AMQ', 'BDJ', 'BDO', 'BKI', 'BKS', 'BOO', 'BPN', 'BTG',
       'BTH', 'BTJ', 'CBN', 'CGK', 'CKR', 'CLG', 'CXP', 'DJB', 'DJJ',
       'DPK', 'DPS', 'DTB', 'GTO', 'JBR', 'JOG', 'KDI', 'KDR', 'KOE',
       'KRW', 'MDC', 'MDN', 'MES', 'MGL', 'MJK', 'MXG', 'PBL', 'PDG',
       'PGK', 'PKU', 'PKY', 'PLM', 'PLW', 'PNK', 'PSR', 'PWT', 'SMD',
       'SMI', 'SOC', 'SOQ', 'SRG', 'SUB', 'TGL', 'TGR', 'TJQ', 'TKG',
       'TNJ', 'TRK', 'TSM', 'TTE', 'UPG'], dtype=object)

In [6]:
print(df['UserVar.branch_dest'].unique().shape)
np.sort(df['UserVar.branch_dest'].unique())

(62,)


array(['AMI', 'AMQ', 'BDJ', 'BDO', 'BKI', 'BKS', 'BOO', 'BPN', 'BTG',
       'BTH', 'BTJ', 'CBN', 'CGK', 'CKR', 'CLG', 'CXP', 'DJB', 'DJJ',
       'DPK', 'DPS', 'DTB', 'GTO', 'JBR', 'JOG', 'KDI', 'KDR', 'KOE',
       'KRW', 'MDC', 'MDN', 'MES', 'MGL', 'MJK', 'MKQ', 'MXG', 'PBL',
       'PDG', 'PGK', 'PKU', 'PKY', 'PLM', 'PLW', 'PNK', 'PSR', 'PWT',
       'SDA', 'SMD', 'SMI', 'SOC', 'SOQ', 'SRG', 'SUB', 'TGL', 'TGR',
       'TIM', 'TJQ', 'TKG', 'TNJ', 'TRK', 'TSM', 'TTE', 'UPG'],
      dtype=object)

there are two more branch in branch_dest, so we can do three things
1. we delete them if there is only a small number of row on the extra two branch
2. or we can add another column of the one hot encoding of the branch and set it to all zero
3. ignore it and assume that the extra branch is for destination branch only

the reason we will use the option number 2 if we will train the model again with more bigger data, we dont need to edit our code again if the total real branch is 62. but if its not both of those reasons then we can do option 1 or 3

i will chose option 3 for this time

In [7]:
# find the values that are in set2 but not in set1
set1 = set(df['UserVar.branch_origin'].unique())
set2 = set(df['UserVar.branch_dest'].unique())

diff = set2.difference(set1)

print(diff)

{'TIM', 'MKQ', 'SDA'}


In [8]:
# encode the branch column
for col in ['UserVar.branch_origin', 'UserVar.branch_dest']:
    one_hot_df = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df, one_hot_df], axis=1)
    df = df.drop(col, axis=1)

In [9]:
df

,taskId,taskCreatedTime,taskStatus,cod.amount,UserVar.receiver_city,UserVar.weight,isCOD,timeDiffMinutes,UserVar.branch_origin_AMI,UserVar.branch_origin_AMQ,UserVar.branch_origin_BDJ,UserVar.branch_origin_BDO,UserVar.branch_origin_BKI,UserVar.branch_origin_BKS,UserVar.branch_origin_BOO,UserVar.branch_origin_BPN,UserVar.branch_origin_BTG,UserVar.branch_origin_BTH,UserVar.branch_origin_BTJ,UserVar.branch_origin_CBN,UserVar.branch_origin_CGK,UserVar.branch_origin_CKR,UserVar.branch_origin_CLG,UserVar.branch_origin_CXP,UserVar.branch_origin_DJB,UserVar.branch_origin_DJJ,UserVar.branch_origin_DPK,UserVar.branch_origin_DPS,UserVar.branch_origin_DTB,UserVar.branch_origin_GTO,UserVar.branch_origin_JBR,UserVar.branch_origin_JOG,UserVar.branch_origin_KDI,UserVar.branch_origin_KDR,UserVar.branch_origin_KOE,UserVar.branch_origin_KRW,UserVar.branch_origin_MDC,UserVar.branch_origin_MDN,UserVar.branch_origin_MES,UserVar.branch_origin_MGL,UserVar.branch_origin_MJK,UserVar.branch_origin_MXG,UserVar.branch_origin_PBL,UserVar.branch_origin_PDG,UserVar.branch_origin_PGK,UserVar.branch_origin_PKU,UserVar.branch_origin_PKY,UserVar.branch_origin_PLM,UserVar.branch_origin_PLW,UserVar.branch_origin_PNK,UserVar.branch_origin_PSR,UserVar.branch_origin_PWT,UserVar.branch_origin_SMD,UserVar.branch_origin_SMI,UserVar.branch_origin_SOC,UserVar.branch_origin_SOQ,UserVar.branch_origin_SRG,UserVar.branch_origin_SUB,UserVar.branch_origin_TGL,UserVar.branch_origin_TGR,UserVar.branch_origin_TJQ,UserVar.branch_origin_TKG,UserVar.branch_origin_TNJ,UserVar.branch_origin_TRK,UserVar.branch_origin_TSM,UserVar.branch_origin_TTE,UserVar.branch_origin_UPG,UserVar.branch_dest_AMI,UserVar.branch_dest_AMQ,UserVar.branch_dest_BDJ,UserVar.branch_dest_BDO,UserVar.branch_dest_BKI,UserVar.branch_dest_BKS,UserVar.branch_dest_BOO,UserVar.branch_dest_BPN,UserVar.branch_dest_BTG,UserVar.branch_dest_BTH,UserVar.branch_dest_BTJ,UserVar.branch_dest_CBN,UserVar.branch_dest_CGK,UserVar.branch_dest_CKR,UserVar.branch_dest_CLG,UserVar.branch_dest_CXP,UserVar.branch_dest_DJB,UserVar.branch_dest_DJJ,UserVar.branch_dest_DPK,UserVar.branch_dest_DPS,UserVar.branch_dest_DTB,UserVar.branch_dest_GTO,UserVar.branch_dest_JBR,UserVar.branch_dest_JOG,UserVar.branch_dest_KDI,UserVar.branch_dest_KDR,UserVar.branch_dest_KOE,UserVar.branch_dest_KRW,UserVar.branch_dest_MDC,UserVar.branch_dest_MDN,UserVar.branch_dest_MES,UserVar.branch_dest_MGL,UserVar.branch_dest_MJK,UserVar.branch_dest_MKQ,UserVar.branch_dest_MXG,UserVar.branch_dest_PBL,UserVar.branch_dest_PDG,UserVar.branch_dest_PGK,UserVar.branch_dest_PKU,UserVar.branch_dest_PKY,UserVar.branch_dest_PLM,UserVar.branch_dest_PLW,UserVar.branch_dest_PNK,UserVar.branch_dest_PSR,UserVar.branch_dest_PWT,UserVar.branch_dest_SDA,UserVar.branch_dest_SMD,UserVar.branch_dest_SMI,UserVar.branch_dest_SOC,UserVar.branch_dest_SOQ,UserVar.branch_dest_SRG,UserVar.branch_dest_SUB,UserVar.branch_dest_TGL,UserVar.branch_dest_TGR,UserVar.branch_dest_TIM,UserVar.branch_dest_TJQ,UserVar.branch_dest_TKG,UserVar.branch_dest_TNJ,UserVar.branch_dest_TRK,UserVar.branch_dest_TSM,UserVar.branch_dest_TTE,UserVar.branch_dest_UPG
0,4fe3b237c832ca4841a2,2022-11-01 13:17:26+00:00,done,685000.0,"BATANG ,KAB BATANG",13.000,1,29.066667,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,08a4da25256affae8446,2022-11-01 01:41:07+00:00,done,53500.0,"PURWODADI,PURWOREJO",1.300,1,232.683333,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2ff0dc469826158b7684,2022-11-01 01:41:07+00:00,done,179500.0,"PURWODADI,PURWOREJO",3.000,1,300.833333,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

# Encode Receiver city

so receiver city has a lot of unique value, so encoding it using one hot encoding could lead to computational issues in theory. so i know its maybe not the right approach to user ordinal encoding, but its better than having a risk of computational issue if the model was deployed to have a high rate usage which is what i assume to be.

so for this time we will use ordinal encoding until we got another type of data that is better suited for receiver city that is available when task status has a value of ongoing

In [10]:
df['UserVar.receiver_city'].unique().shape

(1788,)

In [11]:
import pickle
from sklearn.preprocessing import OrdinalEncoder

# create an instance of OrdinalEncoder and fit on the 'color' column
encoder = OrdinalEncoder()
df['UserVar.receiver_city'] = encoder.fit_transform(df['UserVar.receiver_city'].values.reshape(-1,1))

# save the fitted encoder using pickle
with open('encoder_receiver_city.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [12]:
df.head()

,taskId,taskCreatedTime,taskStatus,cod.amount,UserVar.receiver_city,UserVar.weight,isCOD,timeDiffMinutes,UserVar.branch_origin_AMI,UserVar.branch_origin_AMQ,UserVar.branch_origin_BDJ,UserVar.branch_origin_BDO,UserVar.branch_origin_BKI,UserVar.branch_origin_BKS,UserVar.branch_origin_BOO,UserVar.branch_origin_BPN,UserVar.branch_origin_BTG,UserVar.branch_origin_BTH,UserVar.branch_origin_BTJ,UserVar.branch_origin_CBN,UserVar.branch_origin_CGK,UserVar.branch_origin_CKR,UserVar.branch_origin_CLG,UserVar.branch_origin_CXP,UserVar.branch_origin_DJB,UserVar.branch_origin_DJJ,UserVar.branch_origin_DPK,UserVar.branch_origin_DPS,UserVar.branch_origin_DTB,UserVar.branch_origin_GTO,UserVar.branch_origin_JBR,UserVar.branch_origin_JOG,UserVar.branch_origin_KDI,UserVar.branch_origin_KDR,UserVar.branch_origin_KOE,UserVar.branch_origin_KRW,UserVar.branch_origin_MDC,UserVar.branch_origin_MDN,UserVar.branch_origin_MES,UserVar.branch_origin_MGL,UserVar.branch_origin_MJK,UserVar.branch_origin_MXG,UserVar.branch_origin_PBL,UserVar.branch_origin_PDG,UserVar.branch_origin_PGK,UserVar.branch_origin_PKU,UserVar.branch_origin_PKY,UserVar.branch_origin_PLM,UserVar.branch_origin_PLW,UserVar.branch_origin_PNK,UserVar.branch_origin_PSR,UserVar.branch_origin_PWT,UserVar.branch_origin_SMD,UserVar.branch_origin_SMI,UserVar.branch_origin_SOC,UserVar.branch_origin_SOQ,UserVar.branch_origin_SRG,UserVar.branch_origin_SUB,UserVar.branch_origin_TGL,UserVar.branch_origin_TGR,UserVar.branch_origin_TJQ,UserVar.branch_origin_TKG,UserVar.branch_origin_TNJ,UserVar.branch_origin_TRK,UserVar.branch_origin_TSM,UserVar.branch_origin_TTE,UserVar.branch_origin_UPG,UserVar.branch_dest_AMI,UserVar.branch_dest_AMQ,UserVar.branch_dest_BDJ,UserVar.branch_dest_BDO,UserVar.branch_dest_BKI,UserVar.branch_dest_BKS,UserVar.branch_dest_BOO,UserVar.branch_dest_BPN,UserVar.branch_dest_BTG,UserVar.branch_dest_BTH,UserVar.branch_dest_BTJ,UserVar.branch_dest_CBN,UserVar.branch_dest_CGK,UserVar.branch_dest_CKR,UserVar.branch_dest_CLG,UserVar.branch_dest_CXP,UserVar.branch_dest_DJB,UserVar.branch_dest_DJJ,UserVar.branch_dest_DPK,UserVar.branch_dest_DPS,UserVar.branch_dest_DTB,UserVar.branch_dest_GTO,UserVar.branch_dest_JBR,UserVar.branch_dest_JOG,UserVar.branch_dest_KDI,UserVar.branch_dest_KDR,UserVar.branch_dest_KOE,UserVar.branch_dest_KRW,UserVar.branch_dest_MDC,UserVar.branch_dest_MDN,UserVar.branch_dest_MES,UserVar.branch_dest_MGL,UserVar.branch_dest_MJK,UserVar.branch_dest_MKQ,UserVar.branch_dest_MXG,UserVar.branch_dest_PBL,UserVar.branch_dest_PDG,UserVar.branch_dest_PGK,UserVar.branch_dest_PKU,UserVar.branch_dest_PKY,UserVar.branch_dest_PLM,UserVar.branch_dest_PLW,UserVar.branch_dest_PNK,UserVar.branch_dest_PSR,UserVar.branch_dest_PWT,UserVar.branch_dest_SDA,UserVar.branch_dest_SMD,UserVar.branch_dest_SMI,UserVar.branch_dest_SOC,UserVar.branch_dest_SOQ,UserVar.branch_dest_SRG,UserVar.branch_dest_SUB,UserVar.branch_dest_TGL,UserVar.branch_dest_TGR,UserVar.branch_dest_TIM,UserVar.branch_dest_TJQ,UserVar.branch_dest_TKG,UserVar.branch_dest_TNJ,UserVar.branch_dest_TRK,UserVar.branch_dest_TSM,UserVar.branch_dest_TTE,UserVar.branch_dest_UPG
0,4fe3b237c832ca4841a2,2022-11-01 13:17:26+00:00,done,685000.0,142.0,13.000,1,29.066667,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,08a4da25256affae8446,2022-11-01 01:41:07+00:00,done,53500.0,1298.0,1.300,1,232.683333,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2ff0dc469826158b7684,2022-11-01 01:41:07+00:00,done,179500.0,1298.0,3.000,1,300.833333,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## taskCreatedTime

task created time will be a unix timestamp or epoch in seconds

In [13]:
df['taskCreatedTime'] = pd.to_datetime(df['taskCreatedTime'],utc=True)

In [14]:
df['taskCreatedTime'] = df['taskCreatedTime'].astype(int) / 10**9

## reorder the columns and save the data

In [15]:
cols = list(df.columns)

In [16]:
cols

['taskId',
 'taskCreatedTime',
 'taskStatus',
 'cod.amount',
 'UserVar.receiver_city',
 'UserVar.weight',
 'isCOD',
 'timeDiffMinutes',
 'UserVar.branch_origin_AMI',
 'UserVar.branch_origin_AMQ',
 'UserVar.branch_origin_BDJ',
 'UserVar.branch_origin_BDO',
 'UserVar.branch_origin_BKI',
 'UserVar.branch_origin_BKS',
 'UserVar.branch_origin_BOO',
 'UserVar.branch_origin_BPN',
 'UserVar.branch_origin_BTG',
 'UserVar.branch_origin_BTH',
 'UserVar.branch_origin_BTJ',
 'UserVar.branch_origin_CBN',
 'UserVar.branch_origin_CGK',
 'UserVar.branch_origin_CKR',
 'UserVar.branch_origin_CLG',
 'UserVar.branch_origin_CXP',
 'UserVar.branch_origin_DJB',
 'UserVar.branch_origin_DJJ',
 'UserVar.branch_origin_DPK',
 'UserVar.branch_origin_DPS',
 'UserVar.branch_origin_DTB',
 'UserVar.branch_origin_GTO',
 'UserVar.branch_origin_JBR',
 'UserVar.branch_origin_JOG',
 'UserVar.branch_origin_KDI',
 'UserVar.branch_origin_KDR',
 'UserVar.branch_origin_KOE',
 'UserVar.branch_origin_KRW',
 'UserVar.branch_origin_

In [17]:
cols.remove('timeDiffMinutes')
cols.remove('taskStatus')

In [18]:
cols.append('timeDiffMinutes')
cols.insert(1,'taskStatus')

In [19]:
cols

['taskId',
 'taskStatus',
 'taskCreatedTime',
 'cod.amount',
 'UserVar.receiver_city',
 'UserVar.weight',
 'isCOD',
 'UserVar.branch_origin_AMI',
 'UserVar.branch_origin_AMQ',
 'UserVar.branch_origin_BDJ',
 'UserVar.branch_origin_BDO',
 'UserVar.branch_origin_BKI',
 'UserVar.branch_origin_BKS',
 'UserVar.branch_origin_BOO',
 'UserVar.branch_origin_BPN',
 'UserVar.branch_origin_BTG',
 'UserVar.branch_origin_BTH',
 'UserVar.branch_origin_BTJ',
 'UserVar.branch_origin_CBN',
 'UserVar.branch_origin_CGK',
 'UserVar.branch_origin_CKR',
 'UserVar.branch_origin_CLG',
 'UserVar.branch_origin_CXP',
 'UserVar.branch_origin_DJB',
 'UserVar.branch_origin_DJJ',
 'UserVar.branch_origin_DPK',
 'UserVar.branch_origin_DPS',
 'UserVar.branch_origin_DTB',
 'UserVar.branch_origin_GTO',
 'UserVar.branch_origin_JBR',
 'UserVar.branch_origin_JOG',
 'UserVar.branch_origin_KDI',
 'UserVar.branch_origin_KDR',
 'UserVar.branch_origin_KOE',
 'UserVar.branch_origin_KRW',
 'UserVar.branch_origin_MDC',
 'UserVar.bran

In [20]:
df = df[cols]

In [21]:
df.head()

,taskId,taskStatus,taskCreatedTime,cod.amount,UserVar.receiver_city,UserVar.weight,isCOD,UserVar.branch_origin_AMI,UserVar.branch_origin_AMQ,UserVar.branch_origin_BDJ,UserVar.branch_origin_BDO,UserVar.branch_origin_BKI,UserVar.branch_origin_BKS,UserVar.branch_origin_BOO,UserVar.branch_origin_BPN,UserVar.branch_origin_BTG,UserVar.branch_origin_BTH,UserVar.branch_origin_BTJ,UserVar.branch_origin_CBN,UserVar.branch_origin_CGK,UserVar.branch_origin_CKR,UserVar.branch_origin_CLG,UserVar.branch_origin_CXP,UserVar.branch_origin_DJB,UserVar.branch_origin_DJJ,UserVar.branch_origin_DPK,UserVar.branch_origin_DPS,UserVar.branch_origin_DTB,UserVar.branch_origin_GTO,UserVar.branch_origin_JBR,UserVar.branch_origin_JOG,UserVar.branch_origin_KDI,UserVar.branch_origin_KDR,UserVar.branch_origin_KOE,UserVar.branch_origin_KRW,UserVar.branch_origin_MDC,UserVar.branch_origin_MDN,UserVar.branch_origin_MES,UserVar.branch_origin_MGL,UserVar.branch_origin_MJK,UserVar.branch_origin_MXG,UserVar.branch_origin_PBL,UserVar.branch_origin_PDG,UserVar.branch_origin_PGK,UserVar.branch_origin_PKU,UserVar.branch_origin_PKY,UserVar.branch_origin_PLM,UserVar.branch_origin_PLW,UserVar.branch_origin_PNK,UserVar.branch_origin_PSR,UserVar.branch_origin_PWT,UserVar.branch_origin_SMD,UserVar.branch_origin_SMI,UserVar.branch_origin_SOC,UserVar.branch_origin_SOQ,UserVar.branch_origin_SRG,UserVar.branch_origin_SUB,UserVar.branch_origin_TGL,UserVar.branch_origin_TGR,UserVar.branch_origin_TJQ,UserVar.branch_origin_TKG,UserVar.branch_origin_TNJ,UserVar.branch_origin_TRK,UserVar.branch_origin_TSM,UserVar.branch_origin_TTE,UserVar.branch_origin_UPG,UserVar.branch_dest_AMI,UserVar.branch_dest_AMQ,UserVar.branch_dest_BDJ,UserVar.branch_dest_BDO,UserVar.branch_dest_BKI,UserVar.branch_dest_BKS,UserVar.branch_dest_BOO,UserVar.branch_dest_BPN,UserVar.branch_dest_BTG,UserVar.branch_dest_BTH,UserVar.branch_dest_BTJ,UserVar.branch_dest_CBN,UserVar.branch_dest_CGK,UserVar.branch_dest_CKR,UserVar.branch_dest_CLG,UserVar.branch_dest_CXP,UserVar.branch_dest_DJB,UserVar.branch_dest_DJJ,UserVar.branch_dest_DPK,UserVar.branch_dest_DPS,UserVar.branch_dest_DTB,UserVar.branch_dest_GTO,UserVar.branch_dest_JBR,UserVar.branch_dest_JOG,UserVar.branch_dest_KDI,UserVar.branch_dest_KDR,UserVar.branch_dest_KOE,UserVar.branch_dest_KRW,UserVar.branch_dest_MDC,UserVar.branch_dest_MDN,UserVar.branch_dest_MES,UserVar.branch_dest_MGL,UserVar.branch_dest_MJK,UserVar.branch_dest_MKQ,UserVar.branch_dest_MXG,UserVar.branch_dest_PBL,UserVar.branch_dest_PDG,UserVar.branch_dest_PGK,UserVar.branch_dest_PKU,UserVar.branch_dest_PKY,UserVar.branch_dest_PLM,UserVar.branch_dest_PLW,UserVar.branch_dest_PNK,UserVar.branch_dest_PSR,UserVar.branch_dest_PWT,UserVar.branch_dest_SDA,UserVar.branch_dest_SMD,UserVar.branch_dest_SMI,UserVar.branch_dest_SOC,UserVar.branch_dest_SOQ,UserVar.branch_dest_SRG,UserVar.branch_dest_SUB,UserVar.branch_dest_TGL,UserVar.branch_dest_TGR,UserVar.branch_dest_TIM,UserVar.branch_dest_TJQ,UserVar.branch_dest_TKG,UserVar.branch_dest_TNJ,UserVar.branch_dest_TRK,UserVar.branch_dest_TSM,UserVar.branch_dest_TTE,UserVar.branch_dest_UPG,timeDiffMinutes
0,4fe3b237c832ca4841a2,done,1.667309e+09,685000.0,142.0,13.000,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,29.066667
1,08a4da25256affae8446,done,1.667267e+09,53500.0,1298.0,1.300,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,232.683333
2,2ff0dc469826158b7684,done,1.667267e+09,179500.0,1298.0,3.000,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [22]:
df.isna().sum()

taskId                       0
taskStatus                   0
taskCreatedTime              0
cod.amount                   0
UserVar.receiver_city        0
                          ... 
UserVar.branch_dest_TRK      0
UserVar.branch_dest_TSM      0
UserVar.branch_dest_TTE      0
UserVar.branch_dest_UPG      0
timeDiffMinutes            742
Length: 129, dtype: int64

In [23]:
df.isna().sum().sum()

742

In [24]:
df.to_csv("data/data-ml-preprocess.csv", index=False)